In [1]:
import sys
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, AutoModel

In [2]:
sys.path.append('/root/tuning_space/Components/')
import interact
from Static import prompt_dict, st, si

In [3]:
model_path = '/root/autodl-fs/weights/SoulChat/'

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
model

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (word_embeddings): Embedding(130528, 4096)
    (layers): ModuleList(
      (0-27): 28 x GLMBlock(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attention): SelfAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
        )
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (mlp): GLU(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)
        )
      )
    )
    (final_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4096, out_features=130528, bias=False)
)

In [ ]:
def chat():
    input_ids = torch.cat([st['[gMASK]'],st['sop'],st['<|system|>'],st['\n']],dim=1).cuda()
    input_ids = torch.cat([input_ids, tokenizer.encode(prompt_dict['system_information'], return_tensors='pt').cuda()[:,2:], st['\n'], st['<|user|>']], dim=1)
    while True:
        input_ids = torch.cat([input_ids, tokenizer.encode(input('User:'), return_tensors='pt').cuda()[:,2:], st['\n'], st['<|assistant|>']], dim=1)
        input_ids = interact.inference_streamed(input_ids, model, tokenizer, in_nature_language=False, greedy=True)

In [ ]:
chat()